In [32]:
#Inicializando o Spark

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark.sql import SparkSession

import pyspark.sql.functions as func

spark = SparkSession.builder.appName("relatorio_shoppings").getOrCreate()

In [33]:
#Importando arquivo excel e definindo schema

from decimal import Decimal
import pandas as pd

cols_ranking_lojas = [{"Loja","Quantidade de Produtos Vendidos","Ticket Médio"}]
pd_file = pd.read_excel("data\\raw\\vendas.xlsx",sheet_name="Plan1",converters={
    "Código Venda":int,
    "Data":str,	
    "ID Loja":str,	
    "Produto":str,	
    "Quantidade":int,	
    "Valor Unitário":Decimal, 
    "Valor Final":Decimal
})

#Criando dataframe spark a partir de um dataframe pandas

df_ranking = spark.createDataFrame(pd_file)
df_ranking.show()

+------------+-------------------+--------------------+--------------------+----------+--------------------+--------------------+
|Código Venda|               Data|             ID Loja|             Produto|Quantidade|      Valor Unitário|         Valor Final|
+------------+-------------------+--------------------+--------------------+----------+--------------------+--------------------+
|           1|2019-01-01 00:00:00|  Iguatemi Esplanada|      Sapato Estampa|         1|358.0000000000000...|358.0000000000000...|
|           1|2019-01-01 00:00:00|  Iguatemi Esplanada|            Camiseta|         2|180.0000000000000...|360.0000000000000...|
|           1|2019-01-01 00:00:00|  Iguatemi Esplanada|       Sapato Xadrez|         1|368.0000000000000...|368.0000000000000...|
|           2|2019-01-02 00:00:00|      Norte Shopping|             Relógio|         3|200.0000000000000...|600.0000000000000...|
|           2|2019-01-02 00:00:00|      Norte Shopping|        Chinelo Liso|         1|71.

In [34]:
# GroupBy e Sum para somar valor de faturamento de cada loja

df_faturamento = df_ranking["ID Loja","Valor Final"].groupby("ID Loja").sum()\
.withColumnRenamed("sum(Valor Final)", "Faturamento")

# Ordenando e renomeando coluna de faturamento

df_faturamento = df_faturamento.sort("Faturamento",ascending=False)
df_faturamento = df_faturamento.withColumn("Faturamento",func.round(df_faturamento["Faturamento"],2))
df_faturamento.show()


+--------------------+-----------+
|             ID Loja|Faturamento|
+--------------------+-----------+
|   Iguatemi Campinas| 1762419.00|
| Shopping Vila Velha| 1731167.00|
| Bourbon Shopping SP| 1726110.00|
|      Rio Mar Recife| 1722766.00|
|  Shopping SP Market| 1721763.00|
|Palladium Shoppin...| 1721120.00|
|      Norte Shopping| 1711968.00|
|   Ribeirão Shopping| 1707122.00|
|  Iguatemi Esplanada| 1699681.00|
|Rio Mar Shopping ...| 1698430.00|
|Shopping Center L...| 1682870.00|
|Novo Shopping Rib...| 1678225.00|
|Shopping Iguatemi...| 1674824.00|
|Center Shopping U...| 1668921.00|
|Shopping União de...| 1663770.00|
|      Shopping Barra| 1653103.00|
|Passei das Águas ...| 1649014.00|
|   Shopping Eldorado| 1648111.00|
|   Salvador Shopping| 1646739.00|
|Shopping Center I...| 1640486.00|
+--------------------+-----------+
only showing top 20 rows



In [35]:
# GroupBy e Sum para somar valor de produtos vendidos por cada loja

df_produtos_vendidos = df_ranking["ID Loja","Quantidade"].groupby("ID Loja").sum()\
.withColumnRenamed("sum(Quantidade)", "Produtos Vendidos")

# Ordenando da loja que mais vendeu para a que menos vendeu

df_produtos_vendidos = df_produtos_vendidos.sort("Produtos Vendidos",ascending=False)
df_produtos_vendidos.show()

+--------------------+-----------------+
|             ID Loja|Produtos Vendidos|
+--------------------+-----------------+
| Shopping Vila Velha|             9224|
|Palladium Shoppin...|             9091|
|      Norte Shopping|             9014|
|Shopping Center L...|             8938|
|Rio Mar Shopping ...|             8937|
|   Iguatemi Campinas|             8935|
|  Shopping SP Market|             8927|
| Bourbon Shopping SP|             8863|
|      Rio Mar Recife|             8863|
|   Ribeirão Shopping|             8825|
|Novo Shopping Rib...|             8751|
|Shopping União de...|             8730|
| Shopping Ibirapuera|             8723|
|   Shopping Eldorado|             8719|
|   Salvador Shopping|             8698|
|Shopping Center I...|             8675|
|      Shopping Barra|             8638|
|Shopping Iguatemi...|             8629|
|Center Shopping U...|             8627|
|Passei das Águas ...|             8618|
+--------------------+-----------------+
only showing top

In [42]:

# Criando tabela temporária para realizar Select com SQL

df_ranking.createOrReplaceTempView('df_table')

# Ticket = Valor Final / Quantidade de produtos vendidos

df_ticket = spark.sql(
    
    'select `ID Loja`, `Valor Final` / Quantidade as Ticket from df_table'
    

)

df_ticket = df_ticket.sort("Ticket",ascending=False)

df_ticket.show()

+--------------------+--------------------+
|             ID Loja|              Ticket|
+--------------------+--------------------+
|    Shopping Morumbi|750.0000000000000...|
|Shopping União de...|750.0000000000000...|
|  Shopping SP Market|750.0000000000000...|
|   Ribeirão Shopping|750.0000000000000...|
|      Shopping Barra|750.0000000000000...|
|   Shopping Eldorado|750.0000000000000...|
|Passei das Águas ...|750.0000000000000...|
|Center Shopping U...|750.0000000000000...|
|   Iguatemi Campinas|750.0000000000000...|
|      Norte Shopping|750.0000000000000...|
|Shopping Midway Mall|750.0000000000000...|
|Center Shopping U...|750.0000000000000...|
|   Iguatemi Campinas|750.0000000000000...|
|Shopping Midway Mall|750.0000000000000...|
|Center Shopping U...|750.0000000000000...|
|Palladium Shoppin...|750.0000000000000...|
|      Shopping Barra|750.0000000000000...|
|Rio Mar Shopping ...|750.0000000000000...|
|Parque Dom Pedro ...|750.0000000000000...|
|Shopping Iguatemi...|750.000000